In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
from datetime import date
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm, datasets
import statsmodels.api as sm
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
df = pd.read_csv('/content/gdrive/My Drive/Research/AWS/modify_date_general_purpose.csv')
df_p2 = pd.read_csv('/content/gdrive/My Drive/Research/AWS/Instance_Type_02.csv')
df = df.loc[df.Instance_Family == "General purpose"]
df.head()

Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,3,9,12,14,15,17,19,20,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,EffectiveDates,PricePerUnit,Project_Type,Project_Size,LeaseContractLength,PurchaseOption,Unnamed: 6,OfferingClass,Location,Instance_Type,Current_Generation,Unnamed: 11,Instance_Family,vCPU,Clock_Speed,Memory,Storage,Network_Performance,Operating_System,Tenancy,usageType,Dedicated EBS Throughput,Normalization Size Factor,Processor Features
0,4/30/2017,0.01,NaN,medium,1,No Upfront,NaN,standard,US East (Ohio),m4.xlarge,Yes,NaN,General purpose,4.0,2.4 GHz,16,EBS,High,RHEL,Dedicated,USE2-DedicatedUsage:m4.xlarge,750 Mbps,8.0,Intel AVX; Intel AVX2; Intel Turbo
1,3/31/2018,0.01,NaN,medium,3,Partial Upfront,NaN,convertible,US East (N. Virginia),m4.large,Yes,NaN,General purpose,2.0,2.4 GHz,8,EBS,Moderate,Linux,Dedicated,DedicatedUsage:m4.large,450 Mbps,4.0,Intel AVX; Intel AVX2; Intel Turbo
4,7/1/2019,0.01,NaN,medium,3,All Upfront,NaN,standard,US West (N. California),m3.xlarge,No,NaN,General purpose,4.0,2.5 GHz,15,SSD,High,RHEL,Shared,USW1-BoxUsage:m3.xlarge,NaN,8.0,Intel AVX; Intel Turbo
5,3/1/2019,0.01,NaN,medium,3,All Upfront,NaN,convertible,US East (N. Virginia),m5ad.2xlarge,Yes,NaN,General purpose,8.0,2.5 GHz,32,SSD,Up to 10,Windows,Shared,BoxUsage:m5ad.2xlarge,Upto 2120 Mbps,16.0,"AVX, AVX2, AMD Turbo"
8,10/1/2018,0.01,NaN,medium,1,AllUpfront,NaN,standard,US West (Oregon),a1,Yes,NaN,General purpose,16.0,2.3 GHz,NaN,EBS,NaN,Linux,Reserved,USW2-ReservedHostUsage:a1,NaN,NaN,NaN


In [ ]:
df_p2

,Instance_Type,Project_Type,Project_Size,vCPU,Memory,Network_Performance
0,AWS_T4g,1,small,2,0.5,3
1,AWS_T4g,2,small,2,1.0,3
2,AWS_T4g,5,small,2,2.0,3
3,AWS_T4g,7,small,2,4.0,3
4,AWS_T4g,8,small,2,8.0,3
...,...,...,...,...,...,...
1511,AWS_M5a,12,medium,16,64.0,5
1512,AWS_M5a,12,medium,32,128.0,5
1513,AWS_M5a,12,large,48,192.0,7
1514,AWS_M5a,12,large,64,256.0,8


In [ ]:
df.dtypes

EffectiveDates                object
PricePerUnit                 float64
Project_Type                 float64
Project_Size                  object
LeaseContractLength            int64
PurchaseOption                object
Unnamed: 6                   float64
OfferingClass                 object
Location                      object
Instance_Type                 object
Current_Generation            object
Unnamed: 11                  float64
Instance_Family               object
vCPU                         float64
Clock_Speed                   object
Memory                        object
Storage                       object
Network_Performance           object
Operating_System              object
Tenancy                       object
usageType                     object
Dedicated EBS Throughput      object
Normalization Size Factor    float64
Processor Features            object
dtype: object

In [ ]:
df["Memory"] = df["Memory"].astype("float64")
df["EffectiveDates"] = df["EffectiveDates"].astype("datetime64")
df["Project_Type"] = df["Project_Type"].astype(str)
df["Storage"] = df["Storage"].astype(str)
df["Processor_Archi"] = df["Storage"].astype(str)

columns = ['Instance_Type', 'Project_Type', 'Project_Size', 'vCPU', 'Memory', 'Network_Performance']
df = df.reindex(columns=columns)
df1 = df[columns]

df1['Instance_Type'] = df1['Instance_Type'].replace({'m4.xlarge': 'AWS_M4', 'm4.large': 'AWS_M4', 'm4.2xlarge': 'AWS_M4', 'm4.4xlarge': 'AWS_M4', 'm4.10xlarge': 'AWS_M4', 'm4.16xlarge': 'AWS_M4', 'm4': 'AWS_M4'})

df1['Instance_Type'] = df1['Instance_Type'].replace({'m3.xlarge': 'AWS_M4', 'm3.large': 'AWS_M4', 'm3.2xlarge': 'AWS_M4', 'm3.4xlarge': 'AWS_M4', 'm3.10xlarge': 'AWS_M4', 'm3.16xlarge': 'AWS_M4', 'm3': 'AWS_M4', 'm3.medium': 'AWS_M4'})

df1['Instance_Type'] = df1['Instance_Type'].replace({'m5n.large': 'AWS_M5n', 'm5n.xlarge': 'AWS_M5n','m5n.2xlarge': 'AWS_M5n', 'm5n.4xlarge': 'AWS_M5n','m5n.8large': 'AWS_M5n', 'm5n.12xlarge': 'AWS_M5n', 'm5n.16xlarge': 'AWS_M5n','m5n.24large': 'AWS_M5n', 'm5dn.large': 'AWS_M5n', 'm5dn.xlarge': 'AWS_M5n', 'm5dn.2xlarge': 'AWS_M5n', 'm5dn.4xlarge': 'AWS_M5n', 'm5dn.8xlarge': 'AWS_M5n', 'm5dn.12xlarge': 'AWS_M5n', 'm5dn.16xlarge': 'AWS_M5n', 'm5dn.24xlarge': 'AWS_M5n'})


df1['Instance_Type'] = df1['Instance_Type'].replace({'m5a.large': 'AWS_M5a', 'm5a.xlarge': 'AWS_M5a','m5a.2xlarge': 'AWS_M5a', 'm5a.4xlarge': 'AWS_M5a','m5a.8xlarge': 'AWS_M5a', 'm5a.12xlarge': 'AWS_M5a', 'm5a.16xlarge': 'AWS_M5a','m5a.24xlarge': 'AWS_M5a', 'm5ad.large': 'AWS_M5a', 'm5ad.xlarge': 'AWS_M5a', 'm5ad.2xlarge': 'AWS_M5a', 'm5ad.4xlarge': 'AWS_M5a', 'm5ad.8xlarge': 'AWS_M5a', 'm5ad.12xlarge': 'AWS_M5a', 'm5ad.16xlarge': 'AWS_M5a', 'm5ad.24xlarge': 'AWS_M5a'})


df1['Instance_Type'] = df1['Instance_Type'].replace({'m5.large': 'AWS_M5', 'm5.xlarge': 'AWS_M5','m5.2xlarge': 'AWS_M5', 'm5.4xlarge': 'AWS_M5','m5.8xlarge': 'AWS_M5', 'm5.12xlarge': 'AWS_M5', 'm5.16xlarge': 'AWS_M5','m5.24xlarge': 'AWS_M5a', 'm5.metal': 'AWS_M5', 'm5d.large': 'AWS_M5', 'm5d.xlarge': 'AWS_M5', 'm5d.2xlarge': 'AWS_M5', 'm5d.4xlarge': 'AWS_M5', 'm5d.8xlarge': 'AWS_M5', 'm5d.12xlarge': 'AWS_M5', 'm5d.16xlarge': 'AWS_M5', 'm5d.24xlarge': 'AWS_M5', 'm5d.metal': 'AWS_M5', 'm5': 'AWS_M5', 'm5d': 'AWS_M5'})


df1['Instance_Type'] = df1['Instance_Type'].replace({'t2.nano': 'AWS_T2', 't2.micro': 'AWS_T2','t2.small': 'AWS_T2', 't2.medium': 'AWS_T2','t2.large': 'AWS_T2', 't2.xlarge': 'AWS_T2', 't2.2xlarge': 'AWS_T2'})

df1['Instance_Type'] = df1['Instance_Type'].replace({'t3a.nano': 'AWS_T3a', 't3a.micro': 'AWS_T3a','t3a.small': 'AWS_T3a', 't3a.medium': 'AWS_T3a','t3a.large': 'AWS_T3a', 't3a.xlarge': 'AWS_T3a', 't3a.2xlarge': 'AWS_T3a'})


df1['Instance_Type'] = df1['Instance_Type'].replace({'t3.nano': 'AWS_T3', 't3.micro': 'AWS_T3','t3.small': 'AWS_T3', 't3.medium': 'AWS_T3','t3.large': 'AWS_T3', 't3.xlarge': 'AWS_T3', 't3.2xlarge': 'AWS_T3'})


df1['Instance_Type'] = df1['Instance_Type'].replace({'t4g.nano': 'AWS_T4g', 't4g.micro': 'AWS_T4g','t4g.small': 'AWS_T4g', 't4g.medium': 'AWS_T4g','t4g.large': 'AWS_T4g', 't4g.xlarge': 'AWS_T4g', 't4g.2xlarge': 'AWS_T4g'})


df1['Instance_Type'] = df1['Instance_Type'].replace({'a1.medium': 'AWS_A1', 'a1.large': 'AWS_A1','a1.xlarge': 'AWS_A1', 'a1.2xlarge': 'AWS_A1','a1.4xlarge': 'AWS_A1', 'a1.metal': 'AWS_A1', 'a1': 'AWS_A1'})

df1['Instance_Type'] = df1['Instance_Type'].replace({'m6g.medium': 'AWS_M6g', 'm6g.large': 'AWS_M6g', 'm6g.xlarge': 'AWS_M6g','m6g.2xlarge': 'AWS_M6g', 'm6g.4xlarge': 'AWS_M6g','m6gd.8xlarge': 'AWS_M6g', 'm6gd.12xlarge': 'AWS_M6g', 'm6gd.16xlarge': 'AWS_M6g','m6gd.metal': 'AWS_M6g','m6gd.medium': 'AWS_M6g', 'm6gd.large': 'AWS_M6g', 'm6gd.xlarge': 'AWS_M6g', 'm6gd.2xlarge': 'AWS_M6g', 'm6gd.4xlarge': 'AWS_M6g', 'm6gd.8xlarge': 'AWS_M6g', 'm6gd.12xlarge': 'AWS_M6g', 'm6gd.16xlarge': 'AWS_M6g', 'm6gd.24xlarge': 'AWS_M6g', 'm6gd.metal': 'AWS_M6g', 'm1.small': 'AWS_M6g', 'm1.medium': 'AWS_M6g', 'm1.large': 'AWS_M6g', 'm1.xlarge': 'AWS_M6g'})

df1.head()

,Instance_Type,Project_Type,Project_Size,vCPU,Memory,Network_Performance
0,AWS_M4,nan,medium,4.0,16.0,High
1,AWS_M4,nan,medium,2.0,8.0,Moderate
4,AWS_M4,nan,medium,4.0,15.0,High
5,AWS_M5a,nan,medium,8.0,32.0,Up to 10
8,AWS_A1,nan,medium,16.0,NaN,NaN


In [ ]:
def fixNetworkPerformanceIssue (row):
   if (row['Instance_Type'] == 'AWS_A1') :
      return 5
   if (row['Instance_Type'] == 'AWS_T3a') :
      return 3
   if (row['Instance_Type'] == 'AWS_T3') :
      return 3
   if (row['Instance_Type'] == 'AWS_T4g') :
      return 3   
   if (row['Instance_Type'] == 'AWS_M4') :
      if (row['vCPU'] >= 0 and row['vCPU'] < 4) :
        return 4  
      if (row['vCPU'] >= 4 and row['vCPU'] < 40) :
        return 6
      if (row['vCPU'] >= 40 and row['vCPU'] < 64) :
        return 7    
      if (row['vCPU'] >= 64) :
        return 10    
   if (row['Instance_Type'] == 'AWS_M5a') :
      if (row['vCPU'] >= 0 and row['vCPU'] < 48) :
        return 5
      if (row['vCPU'] >= 48 and row['vCPU'] < 64) :
        return 7
      if (row['vCPU'] >= 64 and row['vCPU'] < 96) :
        return 8    
      if (row['vCPU'] >= 96) :
        return 9 
   if (row['Instance_Type'] == 'AWS_M6g') :
      if (row['vCPU'] >= 0 and row['vCPU'] < 32) :
        return 5 
      if (row['vCPU'] >= 32 and row['vCPU'] < 48) :
        return 8
      if (row['vCPU'] >= 48 and row['vCPU'] < 64) :
        return 9    
      if (row['vCPU'] >= 64) :
        return 10
   if (row['Instance_Type'] == 'AWS_M5') :
      if (row['vCPU'] >= 0 and row['vCPU'] < 32) :
        return 5 
      if (row['vCPU'] >= 32 and row['vCPU'] < 48) :
        return 7 
      if (row['vCPU'] >= 48 and row['vCPU'] < 64) :
        return 8    
      if (row['vCPU'] >= 64 and row['vCPU'] < 96) :
        return 9     
      if (row['vCPU'] >= 96) :
        return 10      
   if (row['Instance_Type'] == 'AWS_T2') :
      if (row['vCPU'] >= 0 and row['vCPU'] < 1) :
        return 1  
      if (row['vCPU'] >= 1 and row['vCPU'] < 4) :
        return 2 
      if (row['vCPU'] >= 4) :
        return 4
   else :
      return row['Network_Performance']       
    

def groupedMemory (row):
   if (row['Memory'] > 0 and row['Memory'] < 1) :
      return 1
   if (row['Memory'] >= 1 and row['Memory'] < 2) :
      return 2  
   if (row['Memory'] >= 2 and row['Memory'] < 4) :
      return 3
   if (row['Memory'] >= 4 and row['Memory'] < 8) :
      return 4
   if (row['Memory'] >= 8 and row['Memory'] < 16) :
      return 5
   if (row['Memory'] >= 16 and row['Memory'] < 32) :
      return 6
   if (row['Memory'] >= 32 and row['Memory'] < 64) :
      return 7
   if (row['Memory'] >= 64 and row['Memory'] < 96) :
      return 8
   if (row['Memory'] >= 96 and row['Memory'] < 128) :
      return 9
   if (row['Memory'] >= 128 and row['Memory'] < 160) :
      return 10 
   if (row['Memory'] >= 160 and row['Memory'] < 256) :
      return 11
   if (row['Memory'] >= 256 and row['Memory'] < 384) :
      return 12
   if (row['Memory'] >= 384) :
      return 13
   else :
      return row['Memory'] 


def groupedvCPU (row):
   if (row['vCPU'] > 0 and row['vCPU'] < 2) :
      return 1
   if (row['vCPU'] >= 2 and row['vCPU'] < 4) :
      return 2
   if (row['vCPU'] >= 4 and row['vCPU'] < 8) :
      return 3
   if (row['vCPU'] >= 8 and row['vCPU'] < 12) :
      return 4
   if (row['vCPU'] >= 12 and row['vCPU'] < 16) :
      return 5
   if (row['vCPU'] >= 16 and row['vCPU'] < 24) :
      return 6 
   if (row['vCPU'] >= 24 and row['vCPU'] < 32) :
      return 7
   if (row['vCPU'] >= 32 and row['vCPU'] < 48) :
      return 8 
   if (row['vCPU'] >= 48 and row['vCPU'] < 64) :
      return 9
   if (row['vCPU'] >= 64 and row['vCPU'] < 72) :
      return 10  
   if (row['vCPU'] >= 72 and row['vCPU'] < 96) :
      return 11
   if (row['vCPU'] >= 96) :
      return 12
   else :
      return row['vCPU'] 

def setMemory (row):
   if(pd.isnull(row['Memory'])) :
       if (row['vCPU'] == 1) :
          return 2
       if (row['vCPU'] == 2) :
          return 3
       if (row['vCPU'] == 3) :
          return 6
       if (row['vCPU'] == 4) :
          return 7
       if (row['vCPU'] == 5) :
          return 7
       if (row['vCPU'] == 6) :
          return 8  
       if (row['vCPU'] == 7) :
          return 8
       if (row['vCPU'] == 8) :
          return 10 
       if (row['vCPU'] == 9) :
          return 11
       if (row['vCPU'] == 10) :
          return 11
       if (row['vCPU'] == 11) :
          return 12
       if (row['vCPU'] == 12) :
          return 13          
   else :
       return row['Memory'] 


def setCPU (row):
   if(pd.isnull(row['vCPU'])) : 
       if (row['Memory'] == '0-1') :
          return '1-2'
       if (row['Memory'] == '1-2') :
          return '1-2'  
       if (row['Memory'] == '2-4') :
          return '1-2'
       if (row['Memory'] == '4-8') :
          return '2-4'
       if (row['Memory'] == '8-16') :
          return '2-4'
       if (row['Memory'] == '16-32') :
          return '4-8'
       if (row['Memory'] == '32-64') :
          return '12-16'
       if (row['Memory'] == '64-96') :
          return '24-32'
       if (row['Memory'] == '96-128') :
          return '48-64'
       if (row['Memory'] == '128-160') :
          return '32-48' 
       if (row['Memory'] == '160-256') :
          return '48-64'
       if (row['Memory'] == '256-384') :
          return '72-96'
       if (row['Memory'] == '384') :
          return '96' 
   else :
       return row['vCPU'] 

def setStorage (row):
       if (row['Instance_Type'] == 'AWS_A1' or row['Instance_Type'] == 'AWS_T4g' or row['Instance_Type'] == 'AWS_T3' or row['Instance_Type'] == 'AWS_T3a' or row['Instance_Type'] == 'AWS_T2' or row['Instance_Type'] == 'AWS_M4') :
          return 'EBS'
       if (row['Storage'] == '1 x 410' or row['Storage'] == '1 x 160' or row['Storage'] == '1 x 420' or row['Storage'] == '2 x 420') :   
          return 'SSD'
       else :
          return row['Storage']  

def setStorageSize (row):
       if (row['Storage'] == 'EBS') :
          return 1
       else :
          return 2              

def setProject_Size (row):
       if (row['Project_Size'] == 'small') :
          return 1
       if (row['Project_Size'] == 'medium') :
          return 2  
       if (row['Project_Size'] == 'large') :
          return 3     

def setProcessor_Archi (row):
       if (row['Instance_Type'] == 'AWS_A1' or row['Instance_Type'] == 'AWS_T4g' or row['Instance_Type'] == 'AWS_M6g') :
          return 2
       else :
          return 1                

def setProjectType (row):
   if (row['Instance_Type'] == 'AWS_A1' and row['vCPU'] == 1) :
      return 1
   if row['Instance_Type'] == 'AWS_A1' and row['vCPU'] == 2 :
      return 4
   if row['Instance_Type'] == 'AWS_A1' and row['vCPU'] == 3 :
      return 3
   if row['Instance_Type'] == 'AWS_A1' and (row['vCPU'] == 4 or row['vCPU'] == 5) :
      return 4
   if row['Instance_Type'] == 'AWS_A1' and row['vCPU'] > 5 :
      return 7
   if row['Instance_Type'] == 'AWS_A1' :
      return 7
   if row['Instance_Type'] == 'AWS_T4g' and row['vCPU'] == 1 and row['Memory'] <= 2 :
      return 1
   if row['Instance_Type'] == 'AWS_T4g' and (row['vCPU'] <= 2) and row['Memory'] <= 2 :
      return 2
   if row['Instance_Type'] == 'AWS_T4g' and (row['vCPU'] <= 2) and row['Memory'] <= 3 :
      return 5
   if row['Instance_Type'] == 'AWS_T4g' and (row['vCPU'] <= 2) and row['Memory'] <= 4 :
      return 7
   if row['Instance_Type'] == 'AWS_T4g' and (row['vCPU'] <= 2) and row['Memory'] <= 5 :
      return 8
   if row['Instance_Type'] == 'AWS_T4g' and row['vCPU'] <= 3 and row['Memory'] <= 6 :
      return 9
   if row['Instance_Type'] == 'AWS_T4g' and (row['vCPU'] > 3) and row['Memory'] > 6 :
      return 10
   if row['Instance_Type'] == 'AWS_T4g' :
      return 10
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] <= 2 and row['Memory'] == 1 :
      return 1
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] <= 2 and row['Memory'] <= 2 :
      return 2
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] <= 2 and row['Memory'] <= 3 :
      return 5
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] <= 2 and row['Memory'] <= 4 :
      return 7
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] <= 2 and row['Memory'] <= 5 :
      return 8
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] <= 3 and row['Memory'] <= 6 :
      return 9
   if row['Instance_Type'] == 'AWS_T3' and row['vCPU'] > 3 and row['Memory'] > 6 :
      return 10
   if row['Instance_Type'] == 'AWS_T3' :
      return 10
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] <= 2 and row['Memory'] == 1 :
      return 1
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] <= 2 and row['Memory'] <= 2 :
      return 2
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] <= 2 and row['Memory'] <= 3 :
      return 5
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] <= 2 and row['Memory'] <= 4 :
      return 7
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] <= 3 and row['Memory'] <= 5 :
      return 8
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] <= 3 and row['Memory'] <= 6 :
      return 9
   if row['Instance_Type'] == 'AWS_T3a' and row['vCPU'] > 3 and row['Memory'] > 6 :
      return 10
   if row['Instance_Type'] == 'AWS_T3a' :
      return 10
   if row['Instance_Type'] == 'AWS_T2' and row['vCPU'] == 1 and row['Memory'] == 1 :
      return 1
   if row['Instance_Type'] == 'AWS_T2' and row['vCPU'] == 1 and row['Memory'] <= 2 :
      return 2
   if row['Instance_Type'] == 'AWS_T2' and row['vCPU'] == 1 or row['vCPU'] <= 4 :
      return 7
   if row['Instance_Type'] == 'AWS_T2' and row['vCPU'] <= 2 and row['Memory'] <= 5 :
      return 11
   if row['Instance_Type'] == 'AWS_T2' and row['vCPU'] <= 3 and row['Memory'] <= 6 :
      return 4
   if row['Instance_Type'] == 'AWS_T2' and row['vCPU'] > 3 and row['Memory'] > 6 :
      return 10
   if row['Instance_Type'] == 'AWS_T2' :
      return 10
   if row['Instance_Type'] == 'AWS_M6g' and row['vCPU'] <= 2 and row['Memory'] <= 5 :
      return 6
   if row['Instance_Type'] == 'AWS_M6g' and row['vCPU'] <= 5 and row['Memory'] <= 7 :
      return 3
   if row['Instance_Type'] == 'AWS_M6g' and row['vCPU'] <= 7 and row['Memory'] <= 9 :
      return 7
   if row['Instance_Type'] == 'AWS_M6g' and row['vCPU'] <= 9 and row['Memory'] <= 11 :
      return 12
   if row['Instance_Type'] == 'AWS_M6g' and row['vCPU'] > 9 and row['Memory'] > 11:
      return 13
   if row['Instance_Type'] == 'AWS_M6g':
      return 13  
   if row['Instance_Type'] == 'AWS_M5' and row['vCPU'] <= 3 and row['Memory'] <= 6 :
      return 6
   if row['Instance_Type'] == 'AWS_M5' and row['vCPU'] <= 8  and row['Memory'] <= 7 :
      return 5
   if row['Instance_Type'] == 'AWS_M5' and row['vCPU'] <= 9 and row['Memory'] <= 11 :
      return 12
   if row['Instance_Type'] == 'AWS_M5' and row['vCPU'] > 9 and row['Memory'] > 11 :
      return 14
   if row['Instance_Type'] == 'AWS_M5' :
      return 14
   if row['Instance_Type'] == 'AWS_M5a' and row['vCPU'] <= 2 and row['Memory'] <= 5 :
      return 6
   if row['Instance_Type'] == 'AWS_M5a' and row['vCPU'] <= 7 and row['Memory'] <= 7:
      return 5
   if row['Instance_Type'] == 'AWS_M5a' and row['vCPU'] <= 9 and row['Memory'] <= 11 :
      return 12
   if row['Instance_Type'] == 'AWS_M5a' and row['vCPU'] > 9 and row['Memory'] > 11 :
      return 14
   if row['Instance_Type'] == 'AWS_M5a' :
      return 14
   if row['Instance_Type'] == 'AWS_M4' and row['vCPU'] <= 5 and row['Memory'] <= 7 :
      return 6
   if row['Instance_Type'] == 'AWS_M4' and row['vCPU'] <= 7 and row['Memory'] <= 10 :
      return 5
   if row['Instance_Type'] == 'AWS_M4' and row['vCPU'] <= 9 and row['Memory'] <= 11 :
      return 12
   if row['Instance_Type'] == 'AWS_M4' and row['vCPU'] > 9 and row['Memory'] > 11 :
      return 14
   if row['Instance_Type'] == 'AWS_M4' :
      return 14

def groupedInstance_Type (row):
   if (row['Instance_Type'] == 'AWS_A1') :
      return 1
   if (row['Instance_Type'] == 'AWS_T4g') :
      return 2
   if (row['Instance_Type'] == 'AWS_T3') :
      return 3   
   if (row['Instance_Type'] == 'AWS_T3a') :
      return 4
   if (row['Instance_Type'] == 'AWS_T2') :
      return 5
   if (row['Instance_Type'] == 'AWS_M4') :   
      return 6
   if (row['Instance_Type'] == 'AWS_M6g') :
      return 7
   if (row['Instance_Type'] == 'AWS_M5') :
      return 8
   if (row['Instance_Type'] == 'AWS_M5a') :
      return 9  
   else :
      return 10

instant_types = {
                 'AWS_A1':1,
                 'AWS_T4g':2,
                 'AWS_T3':3,
                 'AWS_T3a':4,
                 'AWS_T2':5,
                 'AWS_M4':6,
                 'AWS_M6g':7,
                 'AWS_M5':8,
                 'AWS_M5a':9
                 }      

     
df1['Network_Performance'] = df1.apply (lambda row: fixNetworkPerformanceIssue(row), axis=1) 
df1['Memory'] = df1.apply (lambda row: groupedMemory(row), axis=1)  
df1['vCPU'] = df1.apply (lambda row: groupedvCPU(row), axis=1)  

df1['Memory'] = df1.apply (lambda row: setMemory(row), axis=1)   
df1['Memory'] = df1['Memory'].astype(int)  

df1['Project_Size'] = df1.apply (lambda row: setProject_Size(row), axis=1)   
df1['Project_Type'] = df1.apply (lambda row: setProjectType(row), axis=1)   
# df1['Instance_Type'] = df1.apply (lambda row: groupedInstance_Type(row), axis=1)
df1['Instance_Type'] = df1.Instance_Type.map (instant_types)

df_p2['Memory'] = df_p2.apply (lambda row: groupedMemory(row), axis=1)  
df_p2['vCPU'] = df_p2.apply (lambda row: groupedvCPU(row), axis=1) 
df_p2['Project_Size'] = df_p2.apply (lambda row: setProject_Size(row), axis=1)   
df_p2['Instance_Type'] = df_p2.Instance_Type.map (instant_types)

df1.head(5) 

,Instance_Type,Project_Type,Project_Size,vCPU,Memory,Network_Performance
0,6,7,2,3,6,6
1,6,7,2,2,5,4
4,6,7,2,3,5,6
5,9,7,2,4,7,5
8,1,7,2,6,8,5


In [ ]:
df2 = df1.loc[df1.Instance_Type.isnull()]
df2.head(50000)

df2 = df1.loc[df1['Instance_Type'] == 8].head(30000)
# df3 = df1.loc[df1['Instance_Type'] == 9].head(70000)
df4 = df1.loc[df1['Instance_Type'] == 6].head(10000)
df5 = df1.loc[df1['Instance_Type'] == 3].head(30000)
df6 = df1.loc[df1['Instance_Type'] == 5].head(10000)
# df7 = df1.loc[df1['Instance_Type'] == 4].head(10000)
df8 = df1.loc[df1['Instance_Type'] == 7].head(10000)
df9 = df1.loc[df1['Instance_Type'] == 1].head(10000)
df10 = df2.append([df4 , df5 , df6 , df8 , df9, df_p2])

In [ ]:
df10['Instance_Type'].value_counts()

8    30000
3    15877
6    10000
5    10000
7     3315
1     2171
4      511
2      504
9      501
Name: Instance_Type, dtype: int64

In [ ]:
from sklearn.utils import shuffle
df10 = shuffle(df10)
df10.reset_index(inplace=True, drop=True)

In [ ]:
y = np.asarray(df10['Instance_Type'])
print(y)

X = df10.iloc[:,1:].values
X

[6 8 8 ... 8 8 3]


array([[ 7,  2,  4,  7,  6],
       [12,  3,  8, 10,  7],
       [12,  3,  6,  8,  5],
       ...,
       [12,  3,  6,  8,  5],
       [12,  3,  8, 10,  7],
       [ 8,  2,  2,  5,  3]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
rbf = svm.SVC(kernel='rbf', gamma='auto', C=1, decision_function_shape='ovo').fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)
print(classification_report(y_test, rbf_pred))
print("Accuracy: " , metrics.accuracy_score(y_test, rbf_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       448
           2       1.00      0.50      0.67       103
           3       0.98      1.00      0.99      3171
           4       1.00      1.00      1.00       102
           5       1.00      1.00      1.00      1984
           6       1.00      1.00      1.00      1944
           7       1.00      1.00      1.00       680
           8       1.00      1.00      1.00      6056
           9       1.00      1.00      1.00        88

    accuracy                           1.00     14576
   macro avg       1.00      0.94      0.96     14576
weighted avg       1.00      1.00      1.00     14576

Accuracy:  0.9965010976948409


In [ ]:
poly = svm.SVC(kernel='poly', degree=5, C=1, decision_function_shape='ovo').fit(X_train, y_train)
poly_pred = poly.predict(X_test)
print(classification_report(y_test, poly_pred))
print("Accuracy: " , metrics.accuracy_score(y_test, poly_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       448
           2       1.00      0.50      0.67       103
           3       0.98      1.00      0.99      3171
           4       1.00      0.84      0.91       102
           5       1.00      1.00      1.00      1984
           6       1.00      1.00      1.00      1944
           7       1.00      1.00      1.00       680
           8       1.00      1.00      1.00      6056
           9       1.00      1.00      1.00        88

    accuracy                           1.00     14576
   macro avg       1.00      0.93      0.95     14576
weighted avg       1.00      1.00      0.99     14576

Accuracy:  0.9954034028540066


In [ ]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
linear_pred = linear.predict(X_test)
print(classification_report(y_test, linear_pred))
print("Accuracy: " , metrics.accuracy_score(y_test, linear_pred))

              precision    recall  f1-score   support

           1       0.99      1.00      0.99       448
           2       1.00      0.38      0.55       103
           3       0.98      1.00      0.99      3171
           4       1.00      1.00      1.00       102
           5       1.00      1.00      1.00      1984
           6       0.68      0.79      0.73      1944
           7       0.76      0.76      0.76       680
           8       0.96      0.91      0.93      6056
           9       0.77      0.81      0.79        88

    accuracy                           0.92     14576
   macro avg       0.90      0.85      0.86     14576
weighted avg       0.92      0.92      0.92     14576

Accuracy:  0.9163007683863886


In [ ]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)
sig_pred = sig.predict(X_test)
print(classification_report(y_test, sig_pred))
print("Accuracy: " , metrics.accuracy_score(y_test, sig_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       448
           2       0.00      0.00      0.00       103
           3       0.08      0.13      0.10      3171
           4       0.00      0.00      0.00       102
           5       0.00      0.00      0.00      1984
           6       0.11      0.23      0.15      1944
           7       0.00      0.00      0.00       680
           8       0.52      0.37      0.43      6056
           9       0.00      0.00      0.00        88

    accuracy                           0.21     14576
   macro avg       0.08      0.08      0.08     14576
weighted avg       0.25      0.21      0.22     14576

Accuracy:  0.21453073545554335


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
x1 = [[14, 3, 12, 13,10]]
cus_pred2 = rbf.predict(x1)
cus_pred2[0]

8